In [14]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

# Load train datasets
train_proteins = pd.read_csv("./input/train_proteins.csv")
train_clinical = pd.read_csv("./input/train_clinical_data.csv")
train_peptides = pd.read_csv("./input/train_peptides.csv")
supplemental_clinical = pd.read_csv("./input/supplemental_clinical_data.csv")

# Load test datasets
test_clinical = pd.read_csv("./example_test_files/test.csv")
test_proteins = pd.read_csv("./example_test_files/test_proteins.csv")
test_peptides = pd.read_csv("./example_test_files/test_peptides.csv")

In [15]:
# Pivot the proteins and peptides tables so each protein and peptide is a feature
train_proteins = train_proteins.pivot_table(values="NPX", index="visit_id", columns="UniProt")
train_peptides = train_peptides.pivot_table(values="PeptideAbundance", index="visit_id", columns="Peptide")

# Merge the three tables
train = train_clinical.merge(train_proteins, on="visit_id", how="left")  \
                      .merge(train_peptides, on="visit_id", how="left")
                      
# Set the patient id as index
train = train.set_index(["patient_id"])

# Drop the visit id column
train = train.drop("visit_id", axis=1)

In [16]:
# Pivot the proteins and peptides tables so each protein and peptide is a feature
test_proteins = test_proteins.pivot_table(values="NPX", index="patient_id", columns="UniProt")
test_peptides = test_peptides.pivot_table(values="PeptideAbundance", index="patient_id", columns="Peptide")

# Set the patient id as index
test_clinical = test_clinical.set_index(["patient_id"])

# Merge the three tables
test = test_proteins.merge(test_peptides, on="patient_id", how="left")

In [17]:
# Create a list of common columns
train_cols = train.columns.values[6:]
test_cols = test.columns.values
cols = list(set(train_cols) & set(test_cols))

# Use only common columns
train = train[list(train.columns.values[:5]) + cols]
test = test[cols]

In [18]:
# Fill updrs and state of medication values
train.updrs_4 = train.updrs_4.fillna(0).round()

# Create a list of the ids of the patients
patient_id = list(train.index.unique())

# Interpolate the missing data of every patient
for patient in patient_id:
    train.loc[patient] = train.loc[patient].interpolate(method="linear").fillna(method="bfill")
    
# Fill the remaining na values with the mean of the columns
train = train.fillna(train.mean())

In [19]:
# Create a list of the ids of the patients
patient_id_test = list(test.index.unique())

# Interpolate the missing data of every patient
for patient in patient_id_test:
    test.loc[patient] = test.loc[patient].interpolate(method="linear").fillna(method="bfill")

# Fill the remaining na values with the mean of the columns
test = test.fillna(test.mean())

In [20]:
# Normalize all the columns values from the train and the test dataset
for col in cols:
    values_list = np.array(list(train[col]) + list(test[col]))
    train[col] = (train[col] - values_list.min()) / (values_list.max() - values_list.min())
    test[col] = (test[col] - values_list.min()) / (values_list.max() - values_list.min())

In [21]:
# Define a list with symptoms
updrs_cols = ['updrs_1','updrs_2','updrs_3','updrs_4']

In [22]:
# Empty dict for slopes
updrs_slopes = {}

# Slope and intercept for every patient and updrs
for patient in patient_id:
    
    # Create empty dicts of lists
    updrs_slopes[patient] = []
    
    for updrs in updrs_cols:
        
        X = train.loc[patient]['visit_month'].values.reshape(-1, 1)
        y = train.loc[patient][updrs].values.reshape(-1, 1)
        lr = LinearRegression()
        lr.fit(X, y)
        updrs_slopes[patient].append(float(lr.coef_))

# Create df for slopes
slopes_df = pd.DataFrame.from_dict(updrs_slopes, orient="index", columns=[s + "_slope" for s in updrs_cols])

# Merge all the data in the train data frame
train = train.merge(slopes_df, how="left", left_index=True, right_index=True)

In [23]:
# Create dictionaries to store the models
updrs_predictors = {}
updrs_slopes_predictors = {}

# Define X
X = train.iloc[:, 5:-4]

# Train a model per each symptom
for updrs in updrs_cols:
    updrs_predictors[updrs] = XGBRegressor()
    updrs_predictors[updrs].fit(X, train.loc[:,updrs])
    
# Train a model for each slope
for updrs in updrs_cols:
    updrs_slopes_predictors[updrs] = XGBRegressor()
    updrs_slopes_predictors[updrs].fit(X, train.loc[:,(updrs + "_slope")])

In [24]:
print(X.columns.values)

['TNTNVNC(UniMod_4)PIEC(UniMod_4)FMPLDVQADREDSRE' 'P04275'
 'VHVSEEGTEPEAMLQVLGPKPALPAGTEDTAKEDAANRK' ...
 'KC(UniMod_4)SYTEDAQC(UniMod_4)IDGTIEVPK' 'QQTHMLDVMQDHFSR'
 'SELEEQLTPVAEETR']


In [25]:
print

<function print>

In [26]:
print(updrs_predictors["updrs_1"].predict(test.iloc[0].values.reshape(1,-1)))

[6.841286]


In [27]:
print(X.iloc[0].values.reshape(-1,1))

[[0.55713589]
 [0.41135769]
 [0.39578593]
 ...
 [0.77509819]
 [0.59227842]
 [0.44540207]]


In [28]:
print(test.columns.values)

['TNTNVNC(UniMod_4)PIEC(UniMod_4)FMPLDVQADREDSRE' 'P04275'
 'VHVSEEGTEPEAMLQVLGPKPALPAGTEDTAKEDAANRK' ...
 'KC(UniMod_4)SYTEDAQC(UniMod_4)IDGTIEVPK' 'QQTHMLDVMQDHFSR'
 'SELEEQLTPVAEETR']


In [ ]:
# Load sample submission fie
output_doc = pd.read_csv("./example_test_files/sample_submission.csv")
output_doc = output_doc.drop(index=output_doc.index)

#TODO: 
for patient in patient_id_test:
    
    visit_id = test_clinical.loc[patient].visit_id.values[0]
    group_key = test_clinical.loc[patient].group_key.values[0]
    
    for updrs in updrs_cols:
        
        # Updrs values predicted with protein and peptides
        updrs_prediction = updrs_predictors[updrs].predict(test.loc[patient].values.reshape(1,-1))
        
        prediction_id = f'{visit_id}_{updrs}_plus_0_months'
        new_row = {'prediction_id': prediction_id, 'rating': int(updrs_prediction.round()), 'group_key': group_key}
        output_doc = output_doc.append(new_row, ignore_index=True)
        
        # To get the slope
        slope = updrs_slopes_predictors[updrs].predict(test.loc[patient].values.reshape(1,-1))
                
        for month in range(6, 25, 6):
            
            #Compute the evolution at a given month
            updrs_prediction =  (updrs_prediction + slope * month)
            prediction_id = f'{visit_id}_{updrs}_plus_{month}_months'
            new_row = {'prediction_id': prediction_id, 'rating': int(updrs_prediction.round()), 'group_key': group_key}
            output_doc = output_doc.append(new_row, ignore_index=True)

output_doc.to_csv('submission.csv',  index = False)


In [51]:
output_doc.head(30)

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,7,0
1,3342_0_updrs_1_plus_6_months,7,0
2,3342_0_updrs_1_plus_12_months,7,0
3,3342_0_updrs_1_plus_18_months,7,0
4,3342_0_updrs_1_plus_24_months,8,0
5,3342_0_updrs_2_plus_0_months,2,0
6,3342_0_updrs_2_plus_6_months,2,0
7,3342_0_updrs_2_plus_12_months,2,0
8,3342_0_updrs_2_plus_18_months,2,0
9,3342_0_updrs_2_plus_24_months,2,0
